In [25]:
import sys, os, pathlib, re, configparser

In [26]:
assets_dir = pathlib.Path.cwd().parent / "SeaPowerStreamingAssets"
assets_dir  = assets_dir.resolve()

vessels_dir = assets_dir / "vessels"
vessels_dir = vessels_dir.resolve()

vessel_dict = {}

for file in vessels_dir.iterdir():
    if(file.name.split('_', 1)[0] == 'all'):
        if 'all' not in vessel_dict:
            vessel_dict['all'] = {}
        vessel_name = file.name.split('_', 1)[-1].split('.')[0]
        vessel_dict['all'][vessel_name] = file.as_posix()
    elif(len(file.name.split('_', 2)) == 3):
        nation, type, vessel_name = file.name.split('_', 2)
        vessel_name = vessel_name.split('.')[0]
        if nation not in vessel_dict:
            vessel_dict[nation] = {}
        if type not in vessel_dict[nation]:
            vessel_dict[nation][type] = {}
        
        vessel_dict[nation][type][vessel_name] = file.as_posix()
    else:
        if 'misc' not in vessel_dict:
            vessel_dict['misc'] = []
        vessel_dict['misc'].append(file.as_posix())

In [ ]:
ini_options = set()
option_blocks = []
def gather_options(ini_path : pathlib.Path):
    setting_block = r"\[[A-Za-z0-9\_]+?\]"
    # section_index = 0
    last_block = ""
    last_sub_block = ""
    with open(ini_path, "r") as f:
        for line in f:
            if len(line) > 0:
                block_match = re.match(setting_block, line)
                if block_match:
                    if block_match.group(0)[-1] == last_block:
                        last_sub_block = block_match.group(0).strip()
                        option_blocks[-1][last_sub_block] = {"name" : last_sub_block}
                    else:
                        last_block = block_match.group(0).strip()
                        option_blocks.append({"name" : last_block})
    print(option_blocks)
                    
            

In [31]:
for nation_k, nation_v in vessel_dict.items():
    if isinstance(nation_v, dict):
        for type_k, type_v in vessel_dict[nation_k].items():
            if isinstance(type_v, dict):
                for vessel_k, vessel_v in vessel_dict[nation_k][type_k].items():
                    gather_options(vessel_v)
            elif isinstance(type_v, list):
                print("list: ", type_v)
                for vessel_name in vessel_dict[nation_k][type_k]:
                    gather_options(vessel_v)
    elif isinstance(nation_v, list):
        for vessel_path in nation_v:
            gather_options(vessel_v)

IndexError: no such group

In [ ]:
sorted_ini_options = sorted(list(ini_options), key=lambda x: x[0])
sorted_ini_options